In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# prefix path
prefix = '/content/drive/MyDrive/데이터캠프/DCC/cv/task1/mini'

In [25]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import datetime

In [22]:
# tup
try: 
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# 데이터 다운로드

In [9]:
# fashion dataset load
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [23]:
# dataset shape
print(train_images.shape, test_images.shape)

(60000, 28, 28) (10000, 28, 28)


# 데이터 전처리


*   x는 255로 나누어 줘서 0에서 1사이로 변환시켜줌
*   y는 원 핫 인코딩을 적용함



In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(train_images, 
                                                  train_labels, 
                                                  test_size = 0.2, 
                                                  random_state=100)

In [14]:
X_train.shape, X_val.shape

((48000, 28, 28), (12000, 28, 28))

In [15]:
# the number of classes
num_class = len(np.unique(Y_val))
num_class

10

*https://github.com/Deep-as-hell-team/Fashion-MNIST-/blob/master/Xception.ipynb 의 코드 참고*

In [6]:
def resize_Xcept(img):
  img = img.reshape(28,28).astype('float32') / 255.0
  img = Image.fromarray(img)
  img = img.resize((84,84), PIL.Image.ANTIALIAS) #리사이징
  img = np.stack((img,)*1, axis=2) #1채널 (3채널로 하면 램 용량초과로 중지됨)
  return img

In [13]:
X_train2 = [resize_Xcept(img) for img in X_train]
X_train2 = np.asarray(X_train2)

X_val2 = [resize_Xcept(img) for img in X_val]
X_val2 = np.asarray(X_val2)

In [16]:
test_images2 = [resize_Xcept(img) for img in test_images]
test_images2 = np.asarray(test_images2)

In [17]:
# make one-hot label
Y_train = keras.utils.to_categorical(Y_train)
Y_val = keras.utils.to_categorical(Y_val)
test_labels = keras.utils.to_categorical(test_labels)

# 모델
---
Xception: 구글이 발표한 cnn모델




In [18]:
base = Xception(weights = None, include_top = False, input_shape=(84,84,1))

In [19]:
model = keras.models.Sequential()
model.add(base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(num_class, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20860904  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 21,045,234
Trainable params: 20,990,706
Non-trainable params: 54,528
_________________________________________________________________


In [26]:
start = time.time()
train_model = model.fit(X_train2, Y_train,
                        batch_size=128,                    
                        epochs = 4,
                        validation_data =(X_val2, Y_val))

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Epoch 1/4
375/375 [==============================] - 258s 688ms/step - loss: 0.2024 - accuracy: 0.9262 - val_loss: 0.2841 - val_accuracy: 0.9093
Epoch 2/4
375/375 [==============================] - 258s 688ms/step - loss: 0.2829 - accuracy: 0.8981 - val_loss: 0.2179 - val_accuracy: 0.9215
Epoch 3/4
375/375 [==============================] - 258s 689ms/step - loss: 0.1930 - accuracy: 0.9298 - val_loss: 0.2221 - val_accuracy: 0.9197
Epoch 4/4
375/375 [==============================] - 258s 689ms/step - loss: 0.1574 - accuracy: 0.9409 - val_loss: 0.1996 - val_accuracy: 0.9273
0:17:13


In [28]:
model.evaluate(test_images2, test_labels)

313/313 [==============================] - 17s 52ms/step - loss: 0.2234 - accuracy: 0.9224


[0.22340178489685059, 0.9223999977111816]